# Possible Process Hollowing Image Loading
Detects Loading of samlib.dll, WinSCard.dll from untypical process e.g. through process hollowing by Mimikatz

## Rule Content
```
- title: Possible Process Hollowing Image Loading
  id: e32ce4f5-46c6-4c47-ba69-5de3c9193cd7
  status: experimental
  description: Detects Loading of samlib.dll, WinSCard.dll from untypical process
    e.g. through process hollowing by Mimikatz
  references:
  - https://cyberwardog.blogspot.com/2017/03/chronicles-of-threat-hunter-hunting-for.html
  author: Markus Neis
  date: 2018/01/07
  tags:
  - attack.defense_evasion
  - attack.t1073
  logsource:
    product: windows
    service: sysmon
    category: null
  detection:
    selection:
      EventID: 7
      Image:
      - '*\notepad.exe'
      ImageLoaded:
      - '*\samlib.dll'
      - '*\WinSCard.dll'
    condition: selection
  falsepositives:
  - Very likely, needs more tuning
  level: high

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-endpoint-winevent-sysmon-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='(event_id:"7" AND process_path.keyword:(*\\notepad.exe) AND module_loaded.keyword:(*\\samlib.dll OR *\\WinSCard.dll))')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()